In [3]:
#data processing
import kagglehub
import pandas as pd
import os
path = kagglehub.dataset_download("mczielinski/bitcoin-historical-data")
files = os.listdir(path)
data = pd.read_csv(f"{path}/{files[0]}")
data.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')
data = data.set_index('timestamp', drop = False)
data.head()

,timestamp,open,high,low,close,volume
timestamp,,,,,,
2012-01-01 10:01:00,2012-01-01 10:01:00,4.58,4.58,4.58,4.58,0.0
2012-01-01 10:02:00,2012-01-01 10:02:00,4.58,4.58,4.58,4.58,0.0
2012-01-01 10:03:00,2012-01-01 10:03:00,4.58,4.58,4.58,4.58,0.0
2012-01-01 10:04:00,2012-01-01 10:04:00,4.58,4.58,4.58,4.58,0.0
2012-01-01 10:05:00,2012-01-01 10:05:00,4.58,4.58,4.58,4.58,0.0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7332157 entries, 2012-01-01 10:01:00 to 2025-12-10 23:57:00
Data columns (total 6 columns):
 #   Column     Dtype         
---  ------     -----         
 0   timestamp  datetime64[ns]
 1   open       float64       
 2   high       float64       
 3   low        float64       
 4   close      float64       
 5   volume     float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 391.6 MB


In [ ]:
# volume 유니크 확인
data.loc[data['volume'] == 0, ['volume']].count()

volume    1310476
dtype: int64

In [17]:
# high ≥ open/close ≥ low 규칙 확인
invalid = data[(data['high'] < data['low']) |
               (data['high'] < data['close']) |
               (data['high'] < data['open']) |
               (data['open'] < data['low']) |
               (data['close'] < data['low'])]
invalid.shape

(0, 6)

1. 전반적 가격 스케일과 분포 수준을 파악

## mean vs median
mean이 극단적으로 높다 -> 분포가 오른쪽으로 꼬리가 길다(양의 왜도, positive skew)

## min max 비율
에버리지가 엄청 크다.

## iqr과 max
.75와 max의 갭이 상당하다.

## std 과 mean
std가 상대적으로 매우 크다.

- 변동성이 매우 높고, 왜도가 양의로 큰것같다.

In [18]:
# 기본 통계량
data[['open','high','low','close','volume']].describe()

,open,high,low,close,volume
count,7.332157e+06,7.332157e+06,7.332157e+06,7.332157e+06,7.332157e+06
mean,2.138595e+04,2.139363e+04,2.137807e+04,2.138596e+04,5.122842e+00
std,3.004907e+04,3.005727e+04,3.004075e+04,3.004907e+04,2.204701e+01
min,3.800000e+00,3.800000e+00,3.800000e+00,3.800000e+00,0.000000e+00
25%,4.438800e+02,4.440000e+02,4.436700e+02,4.438800e+02,1.977006e-02
50%,7.303590e+03,7.307990e+03,7.299820e+03,7.303530e+03,4.491397e-01
75%,3.105398e+04,3.107400e+04,3.103700e+04,3.105487e+04,2.895274e+00
max,1.262020e+05,1.262720e+05,1.261580e+05,1.262020e+05,5.853852e+03


In [21]:
years = sorted(data.index.year.unique())
for y in years:
    print("=" * 80)
    print(f" Year: {y} ")
    print("=" * 80)
    
    cols = data.columns.difference(['timestamp'])
    yearly = data.loc[data.index.year == y, cols]
    print(yearly.describe())
    print("\n")

 Year: 2012 
               close           high            low           open  \
count  526439.000000  526439.000000  526439.000000  526439.000000   
mean        8.234715       8.234894       8.234569       8.234747   
std         3.125876       3.126029       3.125762       3.125911   
min         3.800000       3.800000       3.800000       3.800000   
25%         5.090000       5.090000       5.090000       5.090000   
50%         6.850000       6.850000       6.850000       6.850000   
75%        11.390000      11.390000      11.390000      11.390000   
max        16.000000      16.410000      15.490000      16.410000   

              volume  
count  526439.000000  
mean        1.077722  
std        13.308873  
min         0.000000  
25%         0.000000  
50%         0.000000  
75%         0.000000  
max      2958.477574  


 Year: 2013 
               close           high            low           open  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean 